In [1]:
import numpy as np
import os
import seaborn as sns
import scipy.stats as stat
import pandas as pd
import matplotlib.pyplot as plt
from pydataset import data
import warnings
warnings.filterwarnings("ignore")
import acquire as acq
import prepare as prep
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

In [2]:
telco_query = """
        select * from customers
        join contract_types using (contract_type_id)
        join internet_service_types using (internet_service_type_id)
        join payment_types using (payment_type_id)
        """

telco_df = acq.get_telco_data(telco_query)

telco_df.head(3)

,Unnamed: 0,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,0,2,1,2,0002-ORFBO,Female,0,Yes,Yes,9,...,Yes,Yes,No,Yes,65.6,593.3,No,One year,DSL,Mailed check
1,1,2,1,1,0003-MKNFE,Male,0,No,No,9,...,No,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check
2,2,1,2,1,0004-TLHLJ,Male,0,No,No,4,...,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check


In [3]:
telco_df = telco_df.drop("Unnamed: 0" ,axis=1)

In [4]:
telco_df.head(3)

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,2,0002-ORFBO,Female,0,Yes,Yes,9,Yes,...,Yes,Yes,No,Yes,65.6,593.3,No,One year,DSL,Mailed check
1,2,1,1,0003-MKNFE,Male,0,No,No,9,Yes,...,No,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check
2,1,2,1,0004-TLHLJ,Male,0,No,No,4,Yes,...,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check


In [5]:
#clean my data
telco_df = prep.prep_telco(telco_df)
telco_df.head(3)

,customer_id,monthly_charges,churn,tech_support_no internet service,tech_support_yes,contract_type_one year,contract_type_two year,internet_service_type_fiber optic,internet_service_type_none,payment_type_credit card (automatic),payment_type_electronic check,payment_type_mailed check
0,0002-ORFBO,65.6,No,0,1,1,0,0,0,0,0,1
1,0003-MKNFE,59.9,No,0,0,0,0,0,0,0,0,1
2,0004-TLHLJ,73.9,Yes,0,0,0,0,1,0,0,1,0


In [6]:
telco_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 12 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   customer_id                           7043 non-null   object 
 1   monthly_charges                       7043 non-null   float64
 2   churn                                 7043 non-null   object 
 3   tech_support_no internet service      7043 non-null   uint8  
 4   tech_support_yes                      7043 non-null   uint8  
 5   contract_type_one year                7043 non-null   uint8  
 6   contract_type_two year                7043 non-null   uint8  
 7   internet_service_type_fiber optic     7043 non-null   uint8  
 8   internet_service_type_none            7043 non-null   uint8  
 9   payment_type_credit card (automatic)  7043 non-null   uint8  
 10  payment_type_electronic check         7043 non-null   uint8  
 11  payment_type_mail